In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from collections import OrderedDict
import sys
import copy
from pathlib import Path

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torchvision.utils import make_grid

sys.path.insert(0,'..')

import src.models.model_helpers as mh
from src.models.data_helpers import load_data, plot_image
from src.models.train_model import train_model
from src.models.predict_model import test, predict

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

%matplotlib inline

In [3]:
torch.__version__

'1.7.0'

In [4]:
data_dir = Path('../data/raw/airlinersnet/')
models_dir = Path("../models/")
batch_size = 16
n_epochs = 1
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [5]:
class_names, datasets, dataloaders = load_data(data_dir, batch_size)

In [ ]:
# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = make_grid(inputs)

#imshow(out, title=[class_names[x] for x in classes])

In [ ]:
model = mh.initialize_model("densenet161", class_names, hidden_units=1104, dropout=0.4, device=device, freeze_model=False)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
model, optimizer = train_model(model, optimizer, dataloaders, criterion, n_epochs, "airliners_net", models_dir)


In [23]:
test(dataloaders, model, criterion, True)

Test Loss: 0.177011


Test Accuracy: 94% (3531/3749)


In [41]:
mh.save_checkpoint(models_dir, "densenet161_retrain_10_epochs", model)

In [42]:
mh.save_full_model(models_dir, "densenet161_retrain_10_epochs_full", model, optimizer, criterion, losses, acc)

In [25]:
test_path = data_dir / "test"
test_img = list(test_path.glob("*/*.jpg"))

In [29]:
predict(test_img[20], model, cat_to_name)

{'Boeing 707': 0.9140122532844543}

In [34]:
caption_image(test_img[20], model, cat_to_name, 5)

{'0': 'Boeing 707', '1': 'Boeing 717', '2': 'Boeing 727', '3': 'Boeing 737', '4': 'Boeing 747', '5': 'Boeing 757', '6': 'Boeing 767', '7': 'Boeing 777', '8': 'Boeing 787', '9': 'ATR 42', '10': 'ATR 72', '11': 'Airbus A220', '12': 'Airbus A300', '13': 'Airbus A310', '14': 'Airbus A320 Family', '15': 'Airbus A330', '16': 'Airbus A340', '17': 'Airbus A350', '18': 'Airbus A380', '19': 'Bombardier DHC-8-400', '20': 'Bombardier CRJ 200 Series', '21': 'Bombardier CRJ 700 Series', '22': 'Embraer ERJ Family', '23': 'Embraer E-Jet Family'}


TypeError: 'int' object is not subscriptable